In [158]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20,20)

In [159]:
#upload data
first_half = pd.read_excel('REdata.xlsx', sheet_name='2023 I half')
second_half = pd.read_excel('REdata.xlsx', sheet_name='2023 II half')
euro_rate_to_rsd = 117.13

In [160]:
#clean data

first_half['Average size of dwellings [square metre]'] = first_half['Average size of dwellings [square metre]'].fillna(first_half['Average size of dwellings [square metre]'].mean())
first_half['Average price per m² [RSD]'] = first_half['Average price per m² [RSD]'].fillna(first_half['Average price per m² [RSD]'].mean())
second_half['Average size of dwellings [square metre]'] = second_half['Average size of dwellings [square metre]'].fillna(first_half['Average size of dwellings [square metre]'].mean())
second_half['Average price per m² [RSD]'] = second_half['Average price per m² [RSD]'].fillna(first_half['Average price per m² [RSD]'].mean())

year_data = [first_half, second_half]



In [161]:
#data modification

#add price for average size dwellings
first_half['Price of average size dwellings']= first_half['Average price per m² [RSD]'] * first_half['Average size of dwellings [square metre]']
second_half['Price of average size dwellings']= second_half['Average price per m² [RSD]'] * second_half['Average size of dwellings [square metre]']


for half in year_data:
    half.rename(columns={'Average price per m² [RSD]': 'Average price per m² [EUR]'}, inplace=True)
    half['Average price per m² [EUR]'] /= euro_rate_to_rsd
    half['Price of average size dwellings'] /= euro_rate_to_rsd
    half['Average price per m² [EUR]'] = half['Average price per m² [EUR]'].round(2)
    half['Price of average size dwellings'] = half['Price of average size dwellings'].round(2)



In [162]:
#combine data
summary_data = pd.concat([first_half, second_half], ignore_index=True).groupby('Teritory').mean().reset_index()

In [163]:
#review data

summary_data.describe()


,Average size of dwellings [square metre],Average price per m² [EUR],Price of average size dwellings
count,169.000000,169.000000,169.000000
mean,55.261031,1254.245325,69688.998846
std,5.331891,423.102095,29608.701101
min,37.000000,430.400000,32310.590000
25%,54.500000,1145.120000,60069.505000
50%,54.985714,1243.510000,68375.210000
75%,54.985714,1243.510000,68375.210000
max,81.500000,4207.355000,294460.670000


In [164]:
summary_data[summary_data['Average price per m² [EUR]'] > summary_data['Average price per m² [EUR]'].mean()]['Average price per m² [EUR]'].describe().round(2)

count      23.00
mean     1944.35
std       775.89
min      1265.01
25%      1372.92
50%      1774.19
75%      2116.09
max      4207.35
Name: Average price per m² [EUR], dtype: float64

In [167]:
def filter_outliers(df: pd.DataFrame):
    mean_price = df['Average price per m² [EUR]'].mean()
    std_price = df['Average price per m² [EUR]'].std()
    filtered_df = df[(df['Average price per m² [EUR]'] <= (mean_price + std_price)) & (df['Average price per m² [EUR]'] > (mean_price - std_price))]
    return filtered_df  


In [168]:
df_without_outliners = filter_outliers(summary_data)